1. Визначте DataFrame з тривимірними векторами слів:

Завантажте модель word embeddings, використовуючи файл, який містить NLP модель.


In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist

import requests
import pickle
import io

## second part
from sklearn import datasets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
url = 'https://drive.google.com/uc?export=download&id=1281E0CDneuKdflWFBUvuyUzujpdGVImz'

# Download the file
response = requests.get(url)

# Load the content into a BytesIO object
with io.BytesIO(response.content) as f:
    word_embeddings = pickle.load(f)

print(f"Number of words: {len(word_embeddings)}")

Number of words: 243


Витягніть тривимірні вектори для всіх слів з цієї моделі.

In [8]:
words = list(word_embeddings.keys())
vectors = np.array(list(word_embeddings.values()))

In [9]:
pca = PCA(n_components=3)
vectors_3d = pca.fit_transform(vectors)

Створіть DataFrame, в якому буде міститися інформація про слова та їхні тривимірні вектори.

In [17]:
words_bag = list(word_embeddings.keys())[30:50] #['oil', 'gas', 'happy', 'sad', 'city', 'town', 'village', 'country', 'continent', 'petroleum', 'joyful']
words_all = list(word_embeddings.keys())

bag_m_d = np.array([vec(word) for word in words_bag]) # Convert each word to its vector representation
print(np.shape(bag_m_d))

all_m_d = np.array([vec(word) for word in words_all]) # Convert each word to its vector representation
print(np.shape(all_m_d))

# Create a dataframe out of the dictionary embedding. This facilitate the algebraic operations
pca = PCA(n_components=3)
embeddings_3d = pca.fit_transform(all_m_d)
print(np.shape(embeddings_3d))

df_all3d = pd.DataFrame(embeddings_3d, index=words_all)

(20, 300)
(243, 300)
(243, 3)


2. Визначте функції для пошуку найближчого слова:

Напишіть функцію, яка приймає тривимірний вектор та знаходить найближче слово в моделі word embeddings_subset.

In [15]:
def word_vec(word, df):
    return np.array(df.loc[word])

def vec(w):
    return word_embeddings[w]

# Define a function to find the closest word to a vector:
def find_closest_word3d(v, df, k = 1):
    """
    Find the closest word(s) in the DataFrame to the given vector.

    :param v: The vector to compare against.
    :param k: The number of closest words to return.
    :return: A list of word(s) that are closest to the given vector.
    """
    # Calculate the vector difference from each word to the input vector
    diff = df.values - v

    # Get the squared L2 norm of each difference vector.
    # It means the squared euclidean distance from each word to the input vector
    delta = np.sum(diff * diff, axis=1)
    # print(delta)
    # Find the index of the minimun distance in the array
    i = np.argmin(delta)
    # Return the row name for this item
    return df.iloc[i].name

Використайте цю функцію для кількох прикладів та переконайтеся, що результати є коректними.

In [24]:
a = word_vec('oil', df_all3d)
b = word_vec('city', df_all3d)
print(find_closest_word3d(a, df_all3d))
print(find_closest_word3d(b, df_all3d))

oil
city


3. Обчисліть векторний добуток для знаходження ортогонального слова:

Виберіть декілька довільних пар слів.
Обчисліть векторний добуток для кожної пари слів та використайте раніше написану функцію для знаходження найближчого слова.
Проаналізуйте результати та спробуйте їх інтерпретувати.

In [35]:
a = word_vec('China', df_all3d)
b = word_vec('city', df_all3d)
c = np.cross(a, b)
find_closest_word3d(c, df_all3d)

'Vaduz'

In [37]:
a = word_vec('China', df_all3d)
b = word_vec('oil', df_all3d)
c = np.cross(a, b)
find_closest_word3d(c, df_all3d)

'Nuuk'

In [38]:
a = word_vec('city', df_all3d)
b = word_vec('oil', df_all3d)
c = np.cross(a, b)
find_closest_word3d(c, df_all3d)

'Belmopan'

Я спробувала використовувати слова China + city, China + oil, city + oil. Результат виходить незрозумілий, я кореляції між вихідними трьома словами не знаходжу.
Здається датасет дуже маленький, чи модель ненатренирована.


4. Напишіть функції визначення кута між словами:

Розробіть функцію, яка обчислює кут між векторами для довільних двох слів.
Протестируйте цю функцію для різних пар слів.
Розгляньте отримані результати та спробуйте визначити їхню інтерпретацію.

In [30]:
def angle_between_vectors(vector1, vector2):
    """
    Calculate the angle in degrees between two vectors.

    :param vector1: The first vector.
    :param vector2: The second vector.
    :return: The angle in degrees between the two vectors.
    """
    # Normalize the vectors
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)

    if norm1 == 0 or norm2 == 0:
        raise ValueError("One or both of the vectors are zero vectors. Cannot compute angle.")

    normalized_vector1 = vector1 / norm1
    normalized_vector2 = vector2 / norm2

    # Compute the cosine of the angle
    cos_theta = np.dot(normalized_vector1, normalized_vector2)

    # Clip the cosine value to the range [-1, 1] to avoid invalid values for arccos
    cos_theta = np.clip(cos_theta, -1.0, 1.0)

    # Determine the angle in radians
    theta_rad = np.arccos(cos_theta)

    # Convert radians to degrees
    theta_deg = np.degrees(theta_rad)

    return theta_deg

In [33]:
vector_a = word_vec('Nuuk', df_all3d)
vector_b = word_vec('China', df_all3d)

angle_degrees = angle_between_vectors(vector_a, vector_b)

print(f"Кут між векторами: {angle_degrees} градусів")

Кут між векторами: 115.06276362630152 градусів


In [34]:
vector_a = word_vec('Nuuk', df_all3d)
vector_b = word_vec('Germany', df_all3d)

angle_degrees = angle_between_vectors(vector_a, vector_b)

print(f"Кут між векторами: {angle_degrees} градусів")

Кут між векторами: 117.38109470457258 градусів


Кут між векторами Nuuk/China   Nuuk/Germany практично однаковий, показує низьку кореляцію, на мою думку.